In [2]:
import os 
infile = "Zn_100_water.relax.in"

## 3차원 구 방식

In [7]:
import os
import numpy as np

# 설정: 원자 파일 이름과 흡착체-금속 거리 기준 (단위: Å)
input_file = infile
output_file = "atomic_positions_adsorption.in"

##################################################
##################################################
##################################################
##################################################
##################################################
adsorption_center = np.array([6.380351  , 5.422767   ,4.385618])  # 흡착 분자의 중심 좌표
relax_radius = 5.5  # Relax할 반경 (XY 평면에서의 거리 기준)
c_limit = 2.0  # C축(Z 방향)으로 이 값 이상이면 Relax하지 않음
##################################################
##################################################
##################################################
##################################################
##################################################

# 파일 읽기
with open(input_file, "r") as f:
    lines = f.readlines()

output_lines = []
for line in lines:
    split_line = line.strip().split()
    if len(split_line) == 4:  # 원자 정보가 있는 줄만 처리
        element, x, y, z = split_line
        coords = np.array([float(x), float(y), float(z)])
        xy_distance = np.linalg.norm(coords[:2] - adsorption_center[:2])  # XY 평면 거리 계산
        z_distance = abs(coords[2] - adsorption_center[2])  # Z 방향 거리 계산

        # Relax 조건 적용:
        # 1) XY 평면에서 relax_radius 이내
        # 2) Z 방향에서 c_limit 이내
        if xy_distance <= relax_radius and z_distance <= c_limit:
            output_lines.append(f"{element}  {x}  {y}  {z}  1  1  1\n")
        else:
            output_lines.append(f"{element}  {x}  {y}  {z}  0  0  0\n")

# 새로운 파일로 저장
with open(output_file, "w") as f:
    f.writelines(output_lines)

print(f"✅ 완료! 새로운 파일 {output_file}이(가) 생성되었습니다.")


✅ 완료! 새로운 파일 atomic_positions_adsorption.in이(가) 생성되었습니다.


## 고정된 분자 시각화 plotly

In [8]:
import numpy as np
import plotly.graph_objects as go
import json

# ATOMIC_POSITIONS 파일 입력
input_file = output_file

# 원소 색상 파일 불러오기
with open("element_colors.json", "r") as f:
    element_colors = json.load(f)

# 원자 데이터 저장 리스트
elements = []
coordinates = []
relax_flags = []

# 파일 읽기
with open(input_file, "r") as f:
    lines = f.readlines()

for line in lines:
    split_line = line.strip().split()
    
    # 7개 값이 있어야 정상적인 원자 데이터로 간주
    if len(split_line) == 7:
        element, x, y, z, rx, ry, rz = split_line
        coordinates.append([float(x), float(y), float(z)])
        elements.append(element)
        relax_flags.append([int(rx), int(ry), int(rz)])

# numpy 배열 변환
coordinates = np.array(coordinates)
relax_flags = np.array(relax_flags)

# Relax 여부 판단 (모든 좌표에서 1 1 1인 경우 Relax로 분류)
is_relaxed = np.all(relax_flags == [1, 1, 1], axis=1)

# 3D 시각화
fig = go.Figure()

# Relaxed 원자 (원소별 색상)
for element in set(elements):  # 중복 없이 원소 리스트 추출
    element_mask = (np.array(elements) == element) & is_relaxed  # 해당 원소 & Relax된 원자만 선택
    if np.any(element_mask):  # 존재하는 경우만 추가
        fig.add_trace(go.Scatter3d(
            x=coordinates[element_mask, 0],
            y=coordinates[element_mask, 1],
            z=coordinates[element_mask, 2],
            mode='markers',
            marker=dict(size=6, color=element_colors.get(element, "black"), opacity=1.0),
            name=f"{element} (Relaxed)"
        ))

# Fixed 원자 (회색)
fig.add_trace(go.Scatter3d(
    x=coordinates[~is_relaxed, 0], 
    y=coordinates[~is_relaxed, 1], 
    z=coordinates[~is_relaxed, 2],
    mode='markers', marker=dict(size=5, color='gray', opacity=0.5),
    name="Fixed Atoms"
))

# 3D 레이아웃 설정
fig.update_layout(
    title="Interactive 3D Atom Visualization (Color by Element)",
    scene=dict(
        xaxis_title="X (Å)",
        yaxis_title="Y (Å)",
        zaxis_title="Z (Å)",
    ),
    margin=dict(l=0, r=0, b=0, t=40),
    height=700
)

# 인터랙티브 플롯 출력
fig.show()


In [9]:
relax_flags

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0,

In [23]:
import plotly
print(plotly.__version__)


5.24.1
